In [1]:
#Librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


In [2]:
#Datos

datos = pd.read_csv('kc_house_data.csv', sep = ',')
datos.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
#Procesamiento de datos

datos_aux = datos
datos['date'] = pd.to_datetime(datos['date'],format = '%Y-%m-%d')
datos.loc[datos['yr_renovated'] != 0, 'yr_renovated'] = pd.to_datetime(datos.loc[datos['yr_renovated'] != 0, 'yr_renovated'], format = '%Y')

datos_aux['yr_renovated_dummy'] = datos['yr_renovated'].apply(lambda x: 1 if x!=0 else 0)
datos_aux = datos_aux.drop(['yr_renovated'], axis=1)
datos_aux['property_age'] = 2022 - datos['yr_built']
columns = ['price','bedrooms', 'bathrooms', 'sqft_living','waterfront', 'view', 'condition', 'grade','sqft_basement','yr_renovated_dummy', 'property_age']
datos_aux = datos_aux[columns]

Y = datos_aux['price']
X= datos_aux
X = X.drop(columns = ['price'])
X.head(10)

,bedrooms,bathrooms,sqft_living,waterfront,view,condition,grade,sqft_basement,yr_renovated_dummy,property_age
0,3,1.00,1180,0,0,3,7,0,0,67
1,3,2.25,2570,0,0,3,7,400,1,71
2,2,1.00,770,0,0,3,6,0,0,89
3,4,3.00,1960,0,0,5,7,910,0,57
4,3,2.00,1680,0,0,3,8,0,0,35
5,4,4.50,5420,0,0,3,11,1530,0,21
6,3,2.25,1715,0,0,3,7,0,0,27
7,3,1.50,1060,0,0,3,7,0,0,59
8,3,1.00,1780,0,0,3,7,730,0,62
9,3,2.50,1890,0,0,3,7,0,0,19


In [13]:
#Modelo

poly = PolynomialFeatures(degree = 3)
poly_X = poly.fit_transform(X)


(21613, 286)

In [5]:
poly_X_train, poly_X_test, poly_Y_train, poly_Y_test = train_test_split(poly_X, Y, test_size = 0.25, random_state = 0)

polynomial_model = LinearRegression().fit(poly_X_train, poly_Y_train)

In [6]:
#Entrenamiento

Y_pred_train = polynomial_model.predict(poly_X_train)
print("MSE: %.2f" % mean_squared_error(poly_Y_train, Y_pred_train, squared =True))
print("RMSE: %.2f" % mean_squared_error(poly_Y_train, Y_pred_train, squared = False))
print("MAE: %.2f" % mean_absolute_error(poly_Y_train, Y_pred_train))
R2 = r2_score(poly_Y_train, Y_pred_train)
print("R^2: %.2f" % R2 )


MSE: 33644223626.95
RMSE: 183423.62
MAE: 125002.73
R^2: 0.75


In [7]:
#Test

Y_pred_test = polynomial_model.predict(poly_X_test)
print("MSE: %.2f" % mean_squared_error(poly_Y_test, Y_pred_test, squared =True))
print("RMSE: %.2f" % mean_squared_error(poly_Y_test, Y_pred_test, squared = False))
print("MAE: %.2f" % mean_absolute_error(poly_Y_test, Y_pred_test))
R2 = r2_score(poly_Y_test, Y_pred_test)
print("R^2: %.2f" % R2 )


MSE: 38541442975.94
RMSE: 196319.75
MAE: 126271.11
R^2: 0.71


In [8]:
#Todo el df

Y_pred_df = polynomial_model.predict(poly_X)
print("MSE: %.2f" % mean_squared_error(Y, Y_pred_df, squared =True))
print("RMSE: %.2f" % mean_squared_error(Y, Y_pred_df, squared = False))
print("MAE: %.2f" % mean_absolute_error(Y, Y_pred_df))
R2 = r2_score(Y, Y_pred_df)
print("R^2: %.2f" % R2 )


MSE: 34868698404.26
RMSE: 186731.62
MAE: 125319.87
R^2: 0.74


In [12]:
X.head(5)

(21613, 10)

In [11]:
#Prediccion

Variables = pd.DataFrame({'bedrooms': [4], 'bathrroms': [2], 'sqft_living': [2500], 'waterfront': [0], 'view': [0], 'condition': [5], 'grade': [7], 'sqft_basement': [500], 'yr_renovated_dummy': [0], 'property_age': [50]})
poly2 = PolynomialFeatures(degree = 3)
poly_Variables = poly.fit_transform(Variables)


Y_prediccion = polynomial_model.predict(poly_Variables)
print("La casa vale: %.2f" % Y_prediccion)


La casa vale: 425344.27
